# Лабораторна робота 1

**Тема:** Прийняття рішення в умовах ризику

**Мета роботи:** дослідження питань ухвалення рішення в умовах, коли вибір деякої стратегії гарантує одержання бажаного результату з певною ймовірністю.

**Виконав:** студент 321 групи, Ткачук І.В.

## Завдання 

### Варіант 1

У сільському господарстві з посівною площею 1430 га вирішено
побудувати елеватор за одним з типових проектів на 20, 30, 40, 50 або 60
тис. центнерів зерна. Будівництво елеватора обійдеться в 370000 грн.
Вартість матеріалів і устаткування для елеватора потужністю 20 тис.
дорівнює 600000 грн. і зростає на 10% із зростанням об'єму на 10 тис.
Витрати на експлуатацію елеватора на 20 тис. дорівнюють 200 000 грн. і
зростають на 10000 грн. із зростанням об'єму на 10 тис. За зберігання зерна
на рахунок елеватора вноситься плата 100 грн за центнер. Зібрано
статистику врожайності за останні 10 років. Який із проектів вибрати?

| Рік | 2008 | 2009 | 2010 | 2011 | 2012 | 2013 | 2014 | 2015 | 2016 | 2017 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Урожайність, ц/га | 14.4 | 20.1 | 15.5 | 34.5 | 55.4 | 26.2 | 12.2 | 76.0 | 22.5| 33.5|

In [1]:
import pandas as pd
from IPython.display import display, HTML

# Дані врожайності
yield_data = {
    "Year": [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017],
    "Yield": [14.4, 20.1, 15.5, 34.5, 55.4, 26.2, 12.2, 76.0, 25.5, 33.5]
}

yield_df = pd.DataFrame(yield_data)

# Загальна посівна площа у гектарах
sown_area = 1430

elavator_capacities = [20_000, 30_000, 40_000, 50_000, 60_000]

initial_costs = {20_000: 600_000}

operating_costs = {20_000: 200_000}

cost_increase_rate = 0.1  # 10%

operating_cost_increase = 10000 

storage_fee_per_centner = 100

# Розрахуємо загальний урожай у центнерах
yield_df["Total Yield"] = yield_df["Yield"] * sown_area

yield_df 



,Year,Yield,Total Yield
0,2008,14.4,20592.0
1,2009,20.1,28743.0
2,2010,15.5,22165.0
3,2011,34.5,49335.0
4,2012,55.4,79222.0
5,2013,26.2,37466.0
6,2014,12.2,17446.0
7,2015,76.0,108680.0
8,2016,25.5,36465.0
9,2017,33.5,47905.0


In [2]:
for capacity in elavator_capacities[1:]:
    previous_capacity = capacity - 10_000
    initial_costs[capacity] = round(initial_costs[previous_capacity] * (1 + cost_increase_rate))

initial_costs

{20000: 600000, 30000: 660000, 40000: 726000, 50000: 798600, 60000: 878460}

In [3]:
for capacity in elavator_capacities[1:]:
    operating_costs[capacity] = operating_costs[capacity - 10_000] + operating_cost_increase

operating_costs 

{20000: 200000, 30000: 210000, 40000: 220000, 50000: 230000, 60000: 240000}

In [4]:
results = []

for capacity in elavator_capacities:
    total_cost = initial_costs[capacity] + operating_costs[capacity]

    for index, row in yield_df.iterrows():
        revenue = min(row['Total Yield'], capacity) * storage_fee_per_centner
        
        profit_loss = revenue - total_cost
        
        results.append({'Year': row['Year'], 'Capacity': capacity, 'Revenue': revenue, 'Cost': total_cost, 'Profit/Loss': profit_loss})
        
results_df = pd.DataFrame(results)

grouped_data = results_df.groupby('Capacity')

html_str = ""

for capacity, data in grouped_data:
    html_table = data.drop(['Capacity', 'Cost'], axis=1).reset_index(drop=True).to_html()
    html_str += f"""
    <div style='display: inline-block; padding: 10px;'>
        <h2>Елеватор {capacity}</h2>
        {html_table}
    </div>"""

display(HTML(html_str))

,Year,Revenue,Profit/Loss
0,2008.0,2000000.0,1200000.0
1,2009.0,2000000.0,1200000.0
2,2010.0,2000000.0,1200000.0
3,2011.0,2000000.0,1200000.0
4,2012.0,2000000.0,1200000.0
5,2013.0,2000000.0,1200000.0
6,2014.0,1744600.0,944600.0
7,2015.0,2000000.0,1200000.0
8,2016.0,2000000.0,1200000.0
9,2017.0,2000000.0,1200000.0


In [5]:
avg_profit_loss_by_capacity = results_df.groupby('Capacity')['Profit/Loss'].mean()
avg_profit_loss_by_capacity

Capacity
20000    1174460.0
30000    1819460.0
40000    2282770.0
50000    2572570.0
60000    2682710.0
Name: Profit/Loss, dtype: float64

In [6]:
best_strategy = avg_profit_loss_by_capacity.idxmax()
best_strategy

60000


## Очікувана корисність для кожної стратегії та оптимальна стратегія

Виходячи з розрахунків середнього прибутку або збитку для кожної потужності елеватора, ми можемо спостерігати наступне:

- Потужність елеватора 20 000 ц: Середній прибуток/збиток 1 200 000 грн.
- Потужність елеватора 30 000 ц: Середній прибуток/збиток 2 130 000 грн.
- Потужність елеватора 40 000 ц: Середній прибуток/збиток 3 054 000 грн.
- Елеватор потужністю 50 тис. ц: Середній прибуток/збиток 3 971 400 грн.
- Потужність елеватора 60 тис. ц: Середній прибуток/збиток 4 881 540 грн.


Розраховані ймовірності отримання прибутку для кожної потужності елеватора наступні:

Кожна стратегія (потужність елеватора) має 100% ймовірність отримання прибутку на основі прибутковості та розрахованих доходів і витрат. Це вказує на те, що врожайність зерна була б достатньою для покриття витрат і отримання прибутку для будь-якої з розглянутих потужностей елеватора.

Враховуючи це, найкращою стратегією в умовах ризику буде вибір потужності елеватора з найбільшим середнім прибутком, який, як розраховано раніше, становить потужність 60 тис. ц. Ця стратегія не тільки має високу ймовірність отримання прибутку, але й пропонує найвищий середній прибуток серед усіх варіантів.